# Trading Strategy Development Project

The goal of our project is to develop an application that allows user to develop, and backtest their own trading strategies, providing them key information about the efficiency and reliability of their strategy

In [4]:
import os
from dotenv import load_dotenv

# Getting backtesting.py to work and implementing their default strategy

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()

bt.plot()

c:\Users\ryanl\anaconda3\envs\PIC16B-25W\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

ValueError: Invalid frequency: ME

In [23]:
print(output)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    94.27374
Equity Final [$]                  56263.51934
Equity Peak [$]                   56309.05934
Commissions [$]                   10563.95154
Return [%]                          462.63519
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                    22.46598
Volatility (Ann.) [%]                 37.4129
CAGR [%]                             14.99343
Sharpe Ratio                          0.60049
Sortino Ratio                          1.1445
Calmar Ratio                           0.6621
Max. Drawdown [%]                   -33.93159
Avg. Drawdown [%]                    -6.16072
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                   93
Win Rate [%]                         54.83871
Best Trade [%]                    

# Creating our own strategy

In [3]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class MACDStrategy(Strategy):
    def init(self):
        # Calculate MACD and Signal Line with adjusted parameters
        self.macd = self.I(self.calculate_macd, self.data.Close, short_window=6, long_window=13)
        self.signal = self.I(self.calculate_signal, self.macd, signal_window=5)

    def calculate_macd(self, series, short_window=6, long_window=13):
        series = pd.Series(series)  # Convert to Pandas Series
        short_ema = series.ewm(span=short_window, adjust=False).mean()
        long_ema = series.ewm(span=long_window, adjust=False).mean()
        return short_ema - long_ema

    def calculate_signal(self, macd, signal_window=5):
        macd = pd.Series(macd)  # Convert to Pandas Series
        return macd.ewm(span=signal_window, adjust=False).mean()

    def next(self):
        if crossover(self.macd, self.signal):
            self.buy(size=1)  # Set a default position size
        elif crossover(self.signal, self.macd):
            self.sell(size=1)  # Set a default position size

# Example usage with sample data
if __name__ == "__main__":
    from backtesting.test import GOOG  # Sample data from backtesting.py library
    
    bt = Backtest(GOOG, MACDStrategy, cash=10000, commission=.002)
    stats = bt.run()
    bt.plot()
    print(stats)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    55.63315
Equity Final [$]                   9598.73916
Equity Peak [$]                    10002.0854
Commissions [$]                     297.09484
Return [%]                           -4.01261
Buy & Hold Return [%]               703.45824
Return (Ann.) [%]                    -0.47931
Volatility (Ann.) [%]                  1.1294
CAGR [%]                             -0.33065
Sharpe Ratio                         -0.42439
Sortino Ratio                        -0.58043
Calmar Ratio                         -0.11886
Max. Drawdown [%]                    -4.03262
Avg. Drawdown [%]                    -1.34932
Max. Drawdown Duration     3101 days 00:00:00
Avg. Drawdown Duration     1035 days 00:00:00
# Trades                                  156
Win Rate [%]                         35.89744
Best Trade [%]                    

# Papertrading API (Alpaca)

In [ ]:
!pip install alpaca-py

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

alpaca_key = os.getenv('ALPACA_KEY')
secret_key = os.getenv('SECRET_KEY')

In [4]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [ ]:
trading_client = TradingClient(alpaca_key, secret_key, paper=True)

# preparing market order
market_order_data = MarketOrderRequest(
                    symbol="TSLA",
                    qty=5,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

market_order = trading_client.submit_order(
                order_data=market_order_data
               )


In [5]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
# Download data
ticker = 'TSLA'
data = yf.download(ticker, period='10y', interval='1d')



[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [ ]:
# Ensure 'Close' column is a Pandas Series (not a DataFrame)
close = data['Close'].values.ravel()


# Compute technical indicators (RSI & Moving Average)
data['RSI'] = talib.RSI(close, timeperiod=14)   
data['MA50'] = talib.SMA(close, timeperiod=50)

# Drop NaN values created by indicators
data.dropna(inplace=True)

# Define crossover logic
data['RSI_Cross_30'] = (data['RSI'] > 40) & (data['RSI'].shift(1) <= 30)  # RSI crossing above 30
data['RSI_Cross_70'] = (data['RSI'] < 60) & (data['RSI'].shift(1) >= 70)  # RSI crossing below 70

# Generate Buy/Sell signals
data['Buy_Signal'] = data['RSI_Cross_30'] & (data['Close'].values.flatten() > data['MA50'].values)
data['Sell_Signal'] = data['RSI_Cross_70'] & (data['Close'].values.flatten() < data['MA50'].values)

# Display last 20 rows
data[data['Buy_Signal'] == True]


In [124]:
(data['RSI_Cross_30'].values & (data['Close'].values.flatten() > data['MA50'].values))

array([False, False, False, ..., False, False, False])

In [120]:
data['RSI_Cross_30'] 

Datetime
2023-02-23 09:30:00    False
2023-02-23 10:30:00    False
2023-02-23 11:30:00    False
2023-02-23 12:30:00    False
2023-02-23 13:30:00    False
                       ...  
2025-02-12 11:30:00    False
2025-02-12 12:30:00    False
2025-02-12 13:30:00    False
2025-02-12 14:30:00    False
2025-02-12 15:30:00    False
Name: RSI_Cross_30, Length: 3445, dtype: bool

In [38]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time

BASE_URL = "https://paper-api.alpaca.markets"

api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')

def get_data(symbol, limit=3630):
    barset = api.get_bars(symbol, timeframe='1Min', limit=limit).df
    df = barset[['close']].copy()
    df['close'] = df['close'].astype(float)
    return df
SYMBOL = 'TSLA'

get_data(SYMBOL)


,close
timestamp,
2025-02-21 09:00:00+00:00,353.7000
2025-02-21 09:01:00+00:00,353.4000
2025-02-21 09:02:00+00:00,353.2800
2025-02-21 09:03:00+00:00,353.3000
2025-02-21 09:04:00+00:00,353.3600
...,...
2025-02-21 17:17:00+00:00,341.6271
2025-02-21 17:18:00+00:00,341.2400
2025-02-21 17:19:00+00:00,340.9700


In [ ]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time

# Alpaca API Credentials
BASE_URL = "https://paper-api.alpaca.markets"

# Initialize Alpaca API
api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')

# Trading Parameters
SYMBOL = "TSLA"
SHORT_WINDOW = 10
LONG_WINDOW = 50
RSI_PERIOD = 10
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
ORDER_SIZE = 1


def get_data(symbol, limit=3630):
    barset = api.get_bars(symbol, timeframe='1Min', limit=limit).df
    df = barset[['close']].copy()
    df['close'] = df['close'].astype(float)
    return df

''' 
def get_data(symbol, limit=100):
    barset = api.get_bars(symbol, timeframe='1Min', limit=limit).df
    print("Fetched Data Columns:", barset.columns)  # Debugging line
    print("Fetched Data Sample:\n", barset.head())  # Debugging line
    return barset

'''


# Function to calculate indicators
def calculate_indicators(df):
    df['SMA_Short'] = df['close'].rolling(window=SHORT_WINDOW).mean()
    df['SMA_Long'] = df['close'].rolling(window=LONG_WINDOW).mean()
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=RSI_PERIOD).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=RSI_PERIOD).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Function to place orders
def place_order(side):
    try:
        api.submit_order(
            symbol=SYMBOL,
            qty=ORDER_SIZE,
            side=side,
            type='market',
            time_in_force='gtc'
        )
        print(f"{side.upper()} order placed for {SYMBOL}")
    except Exception as e:
        print(f"Order failed: {e}")

# Main loop for trading
while True:
    df = get_data(SYMBOL)
    df = calculate_indicators(df)
    latest = df.iloc[-1]
    print(latest['RSI'])

    position = api.get_position(SYMBOL) if SYMBOL in [pos.symbol for pos in api.list_positions()] else None

    if latest['SMA_Short'] > latest['SMA_Long'] and latest['RSI'] < RSI_OVERBOUGHT:
        place_order("buy")
    elif latest['SMA_Short'] < latest['SMA_Long'] and latest['RSI'] > RSI_OVERSOLD and position is not None:
        place_order("sell")
    #if latest['RSI'] < RSI_OVERSOLD and position is None:
     #   place_order("buy")
    #elif latest['RSI'] > RSI_OVERBOUGHT and position is not None:
      #  place_order("sell")
    time.sleep(60)  # Run every minute


51.30349383965257
SELL order placed for TSLA
50.21848054066631
44.59717944561492
46.61154784494362
42.26668132351328
46.66262135922294
48.324256388771694
55.05966587112139


In [20]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time

# Alpaca API Credentials
BASE_URL = "https://paper-api.alpaca.markets"
DATA_URL = "https://data.alpaca.markets/v2"

# Initialize Alpaca API
api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')
data_api = tradeapi.REST(alpaca_key, secret_key, DATA_URL, api_version='v2')

# Trading Parameters
SYMBOL = "SPY"
SHORT_WINDOW = 10
LONG_WINDOW = 50
RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
ORDER_SIZE = 1

def get_historical_data(symbol, start, end):
    try:
        barset = api.get_bars(symbol, timeframe='1Min', start=start, end=end).df
        print("Fetched Data Columns:", barset.columns)  # Debugging line
        print("Fetched Data Sample:\n", barset.head())  # Debugging line
        if barset.empty:
            print("No data fetched for the given symbol and date range.")
            return pd.DataFrame()  # Return an empty DataFrame
        df = barset[['close']].copy()
        df['close'] = df['close'].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame

# Function to calculate indicators
def calculate_indicators(df):
    if df.empty:
        print("Empty DataFrame, skipping indicator calculation.")
        return df
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=RSI_PERIOD).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=RSI_PERIOD).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Function to place orders
def place_order(side):
    try:
        api.submit_order(
            symbol=SYMBOL,
            qty=ORDER_SIZE,
            side=side,
            type='market',
            time_in_force='gtc'
        )
        print(f"{side.upper()} order placed for {SYMBOL}")
    except Exception as e:
        print(f"Order failed: {e}")

# Fetch historical data
start_date = '2020-01-08'
end_date = '2020-01-09'
print(f"Fetching historical data for {SYMBOL} from {start_date} to {end_date}")
df = get_historical_data(SYMBOL, start=start_date, end=end_date)
df = calculate_indicators(df)

# Simulate real-time data updates
if not df.empty:
    for i in range(len(df)):
        latest = df.iloc[i]
        print(f"Index: {latest.name}, RSI: {latest['RSI']}")

        position = api.get_position(SYMBOL) if SYMBOL in [pos.symbol for pos in api.list_positions()] else None

        if latest['RSI'] < RSI_OVERSOLD:
            place_order("buy")
        elif latest['RSI'] > RSI_OVERBOUGHT and position is not None:
            place_order("sell")
        time.sleep(1)  # Simulate waiting for the next minute
else:
    print("No data available to simulate real-time updates.")

Fetching historical data for SPY from 2020-01-08 to 2020-01-09
Fetched Data Columns: Index(['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], dtype='object')
Fetched Data Sample:
                             close    high      low  trade_count    open  \
timestamp                                                                 
2020-01-08 00:00:00+00:00  319.23  319.54  319.175           82  319.48   
2020-01-08 00:01:00+00:00  319.39  319.39  319.260           32  319.30   
2020-01-08 00:02:00+00:00  319.25  319.37  319.250           32  319.34   
2020-01-08 00:03:00+00:00  319.35  319.42  319.350           32  319.37   
2020-01-08 00:04:00+00:00  319.19  319.39  319.170           41  319.39   

                           volume        vwap  
timestamp                                      
2020-01-08 00:00:00+00:00   17216  319.397892  
2020-01-08 00:01:00+00:00    4816  319.320530  
2020-01-08 00:02:00+00:00    8564  319.308250  
2020-01-08 00:03:00+00:00    5747  319

KeyboardInterrupt: 